In [2]:
#Imports
import pyarrow
import feather
import time
import json
import pandas
import csv
import string
import re
import tables
import numpy as np
import pyarrow.feather as pf


from tables import *
from datetime import datetime

In [3]:
# Singling CSV for hydration
# takes in csv file from online dataset and produces a csv of the twitter ids allowing us to rehydrate them

fileString = 'corona_tweets_14.csv' #the first one to be added
hydrateString = 'ids_to_hydrate1.csv' #The first one that will be added
j = 13 #hydrate string - 1

for i in range(14, 44):
    fileString = fileString.replace(str(i-1), str(i)) #
    print(fileString)
    data = pandas.read_csv(fileString, header = None)
    ids = data[0]
    hydrateString = hydrateString.replace(str(j), str(j+1))
    ids.to_csv(hydrateString, header=None, index=None)
    j = j+1

In [ ]:
#This cell reads in each data file and stores the desired contents of each tweet in a data frame. It combines all the
# data for the whole data range (all of April 2020) and saves it as a singe .feather file with the path specified at the
# bottom

#Function that converts the time into a readable format
def convTime(dtime):
    new_datetime = datetime.strftime(datetime.strptime(dtime,'%a %b %d %H:%M:%S +0000 %Y'), '%d-%m-%Y')
    return new_datetime


#Filename of the first dataset file to extract the sentiment values from
fileString = 'corona_tweets_14.csv'
#Filename of the first hydrated file to read from
hydratedString = 'april1'
#Define the master dataframe
megaTable = pandas.DataFrame()


for j in range(1,31): 
    print("day number:" + str(j))
    plotdf = pandas.DataFrame(columns=['tweetid', 'text','date','sentiment']) #temporary dataframe
    i = 0;
    fileString = fileString.replace(str(j+12), str(j+13)) #update dataset file name to read from
    sentiment_data = pandas.read_csv(fileString, header=None)
    hydratedString = "april" + str(j) #update hydrated file name to read from
    
    with open(hydratedString, 'r', encoding="utf8") as json_file:
        json_list = list(json_file)
        
    for json_str in json_list: #For each tweet extract the meaningful data
        result = json.loads(json_str)
        current_sentiment_index = sentiment_data.loc[sentiment_data[0] == int(result['id_str'])].index[0]
        current_sentiment_value = sentiment_data.loc[current_sentiment_index][1]
        plotdf.loc[i] = [result['id_str'] , result['full_text'], convTime(result['created_at']), current_sentiment_value]
        i = i+1
    
    frames = [megaTable, plotdf]
    megaTable = pandas.concat(frames)
    

#Output file for the feather file
path = 'april_data.feather'
feather.write_dataframe(megaTable, path)